In [5]:
from ipyparallel import Client
import os
import pprint
pp = pprint.PrettyPrinter(indent=4)

rc = Client()
data_path = os.path.abspath("data")
res_path = os.path.abspath("results")
random_path = os.path.abspath("random")
rc[:].push(dict(
    data_path=data_path,
    res_path=res_path,
    random_path=random_path
        
))

dview = rc.load_balanced_view()

#dview = rc.direct_view('all')
print 'n of workers:', len(dview)
print len(rc)

n of workers: 4
4


In [4]:
pp.pprint(dview.queue_status())

{   0: {   'completed': 1, 'queue': 0, 'tasks': 0},
    1: {   'completed': 1, 'queue': 0, 'tasks': 0},
    2: {   'completed': 1, 'queue': 0, 'tasks': 0},
    3: {   'completed': 1, 'queue': 0, 'tasks': 0},
    'unassigned': 0}


In [6]:
%%px --local
import os
import pandas as pd
from string import strip
from goatools.base import download_go_basic_obo
from goatools.associations import read_associations
from goatools.go_enrichment import GOEnrichmentStudy
from goatools.obo_parser import GODag
import scipy.cluster.hierarchy as sch
import numpy as np
from string import strip
from itertools import combinations
from scipy import stats

In [ ]:
in_go=os.path.join(data_path,'go-basic.obo')
in_assoc=os.path.join(data_path,'associations.txt')
gene_found = [strip(n.split('\t')[0]) for n in open(os.path.join(data_path,'in_df.txt')).read().split('\n')]
obodag = GODag(in_go)
geneid2gos = read_associations(in_assoc)
goeaobj = GOEnrichmentStudy(
            gene_found, # List of mouse protein-coding genes
            geneid2gos, # geneid/GO associations
            obodag, # Ontologies
            propagate_counts = False,
            alpha = 0.05, # default significance cut-off
            methods = ['fdr_bh'])

In [ ]:
def score(in_genes, index_res, cut_distance, cluster_id, out_folder):
    print out_folder
    def count(goea_results_sig):
            res = {'BP' : 0,
                   'CC' : 0,
                   'MF' : 0}
            for item in goea_results_sig:
                item_list = str(item).split()            
                go_type = item_list[1] 
                res[go_type]+=1
            return res   
    goea_results_all = goeaobj.run_study(in_genes)
    goea_results_sig = [r for r in goea_results_all if r.p_fdr_bh < 0.05]
    res = count(goea_results_sig)
    if sum(res.values())>0:
        file_name = 'res_'+str(cut_distance)+'_'+str(cluster_id)+'.txt'
        res_file = open(os.path.join(out_folder,file_name),'w')
        res_file.write('\t'.join(['cut_distance','cluster_id','CC','MF','BP'])+'\n')
        res_file.write('\t'.join( [str(n) for n in [cut_distance,cluster_id,res['CC'],res['MF'],res['BP']]] ))
        res_file.close()
    #return count(goea_results_sig)

In [ ]:
def test():
    in_file = os.path.join('test','protein_complexes','T_complex.txt')
    geneids_study = [strip(n) for n in open(in_file).read().split('\n')]
    score(geneids_study, 'test_local', 0.04, 2, random_path)
    rc[1].apply(score, geneids_study, 'test_cluster', 0.05, 3, random_path)
test()

In [8]:
import random
clustering_df = pd.DataFrame.from_csv('res_clustering.csv')
print clustering_df.iloc[0:10,0:2].head()
random_df = clustering_df[clustering_df.columns]
index = random_df.index.values
random.shuffle(index)
random_df['index']=index
random_df.set_index('index',inplace=True)
print random_df.iloc[0:10,0:2].head()
print clustering_df.shape
print random_df.shape
#print clustering_df.head()

               clusters_0.0  clusters_0.06
Tb927.10.3210          2325           2324
Tb927.5.2310           1801           1800
Tb927.8.7570           3018           3017
Tb927.3.3880           2704           2703
Tb927.7.5640           2640           2639
                clusters_0.0  clusters_0.06
index                                      
Tb927.10.11210          2325           2324
Tb927.11.3690           1801           1800
Tb927.6.4530            3018           3017
Tb927.9.10350           2704           2703
Tb927.3.1040            2640           2639
(3117, 419)
(3117, 419)


In [ ]:
import time
import sys
start_time = time.time()
#res = []

def process_df(clustering_df=clustering_df, res_path=res_path):
    for index, col in enumerate(clustering_df.columns):
        in_data = clustering_df[col].value_counts()
        in_data = in_data[in_data>=2]
        cut_distance = col.split('_')[-1]
        print index, col, cut_distance
        for cluster in in_data.index.values:
            genes = clustering_df[col][clustering_df[col]==cluster]
            genes = genes.index.values
            #print cluster, len(genes)
            #print 'send', index, cut_distance, cluster, 'to controller'
            dview.apply(score, genes, index, cut_distance, cluster,res_path)
            test = 0
            a=0
            while 1:
                #send more job only if the queue is less than 100
                #help to do not run out of memory
                if dview.queue_status()['unassigned']<100:            
                    break
                else:
                    a+=1
                    #print a, 'unassigned:', dview.queue_status()['unassigned']
                    #print 'waited for ', a*1, 'sec'
                    sys.stdout.flush()
                    time.sleep(1)            
                    pass        

def test():
    process_df(clustering_df=clustering_df.iloc[:,10:20], res_path=res_path)
    process_df(clustering_df=random_df.iloc[:,10:20], res_path=random_path)
#test()
process_df(clustering_df=clustering_df, res_path=res_path)
process_df(clustering_df=random_df, res_path=random_path)

print 'end after : ', time.time() - start_time

In [ ]:
pp.pprint(dview.queue_status())

In [7]:
def reduce_res(res_path, outname):
    list_df = [pd.read_table(os.path.join(res_path, file_name),
                             sep='\t') for file_name in os.listdir(res_path)]
    list_df = pd.concat(list_df)
    list_df.sort_values(by='cut_distance', ascending=True, inplace=True)
    print list_df.head()
    combined = list_df.groupby('cut_distance').sum()
    del combined['cluster_id']
    print combined.head()
    combined.to_csv(outname)

In [11]:
import os
import pandas as pd
reduce_res(os.path.abspath("results"), 'reduced_go_terms.csv')
reduce_res(os.path.abspath("random"), 'reduced_random_go_terms.csv')

   cut_distance  cluster_id  CC  MF  BP
0          0.18        2869   0   1   1
0          0.19        2856   0   1   1
0          0.20        2841   0   1   1
0          0.21        2830   0   1   1
0          0.22        2822   0   1   1
              CC  MF  BP
cut_distance            
0.18           0   1   1
0.19           0   1   1
0.20           0   1   1
0.21           0   1   1
0.22           0   1   1
